# Toronto Neighborhoods
Author:    Guy Shavit
part 1 scrape the Wikipedia page in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe 

Importing packages

In [40]:
import pandas as pd

Get the Wikipedia page and read into a Dataframe

In [41]:
!wget -O List_of_postal_codes_of_Canada:_M https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
df = pd.read_html('List_of_postal_codes_of_Canada:_M')

--2021-02-12 13:58:41--  https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
Resolving en.wikipedia.org (en.wikipedia.org)... 208.80.153.224, 2620:0:860:ed1a::1
Connecting to en.wikipedia.org (en.wikipedia.org)|208.80.153.224|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 54740 (53K) [text/html]
Saving to: ‘List_of_postal_codes_of_Canada:_M’

List_of_postal_code 100%[===================>]  53.46K  --.-KB/s    in 0.003s  

2021-02-12 13:58:41 (16.7 MB/s) - ‘List_of_postal_codes_of_Canada:_M’ saved [54740/54740]



Take the first table from the Wikipedia page into a Dataframe, Drop any line with Boroguh "Not assigned", reset the index and set the Neighbourhood to the Borough name for any Neighbourhood that is "Not assigned"

In [44]:
PostCode = df[0]
PostCode.drop(PostCode.loc[PostCode['Borough']=="Not assigned"].index, inplace=True)
PostCode.reset_index(drop=True, inplace=True)
PostCode.loc[PostCode.Neighbourhood == "Not assigned", "Neighbourhood"] = PostCode.Borough
PostCode.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Get the table shape

In [45]:
PostCode.shape

(103, 3)